In [1]:
from __future__ import print_function
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
import numpy as np
from random import shuffle
from random import randrange
import time 

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

In [48]:
### Load the raw CIFAR-10 data.
cifar10_dir = '/Users/xiaoxiaoma/cifar-10-batches-py'

# Cleaning up variables to prevent loading data multiple times (which \
# may cause memory issue)
try:
    del X_train, y_train
    del X_test, y_test
    print('Clear previously loaded data.')
except:
    pass
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)


### Subsample the data for more efficient code execution in this exercise
mask = 0
num_train = 500
num_test = 100
num_val = 100
num_dev = 50

mask = range(num_train, num_train + num_val)
X_val = X_train[mask]
y_val = y_train[mask]

mask = range(num_train)
X_train = X_train[mask]
y_train = y_train[mask]

mask = np.random.choice(num_train, num_dev, replace=False)
X_dev = X_train[mask]
y_dev = y_train[mask]

mask = range(num_test)
X_test = X_test[mask]
y_test = y_test[mask]


### Reshape the image data into rows
# print (X_train.shape[0])
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_val = np.reshape(X_val, (X_val.shape[0], -1))
X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))


### Preprocessing: subtract the mean image
# first: compute the image mean based on the training data
mean_image = np.mean(X_train, axis=0)
# print(mean_image[:10]) # print a few of the elements
# plt.figure(figsize=(4,4))
# plt.imshow(mean_image.reshape((32,32,3)).astype('uint8')) 
# visualize the mean image
# plt.show()

# second: subtract the mean image from train and test data
X_train -= mean_image
X_val -= mean_image
X_test -= mean_image
X_dev -= mean_image

# third: append the bias dimension of ones (i.e. bias trick) so that our 
# SVM only has to worry about optimizing a single weight matrix W.
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

print(X_train.shape, X_val.shape, X_test.shape, X_dev.shape)

# generate a random SVM weight matrix of small numbers, W
def init_W(X, y):
    W = np.random.rand(X.shape[1], np.max(y) + 1) * 0.0001
    return W

Clear previously loaded data.
(500, 3073) (100, 3073) (100, 3073) (50, 3073)


In [46]:
# Loss functions

def svm_loss_naive(X, y, W, reg):
    """
    Structured SVM loss function, naive implementation (with loops).
    unvectorized. Compute the multiclass svm loss for a single example(x,y)
    
    - x is a column vector representing an image (e.g. 3073 x 1 in 
    CIFAR-10)with an appended bias dimension in the 3073-rd position 
    (i.e. bias trick)
    - y is an integer giving index of correct class (e.g. between 0 and 9)
    - W is the weight matrix (e.g. 10 x 3073 in CIFAR-10)
  
    Inputs:

    - X: A numpy array of shape (N, D) containing a minibatch of data.
    - y: A numpy array of shape (N,) containing training labels, (0,C); 
      y[i] = c means that X[i] has label c, where 0 <= c < C.
    - W: A numpy array of shape (D, C) containing weights.
    - reg: (float) regularization strength

    returns:
    - loss as single float
    - gradient with respect to weights W; an array of same shape as W 
    """
    
    dW = np.zeros(W.shape) # initialize the gradient as zero
    
    delta = 1.0
    num_classes = W.shape[1]
    num_train = X.shape[0]
    loss = 0.0
    
    for i in range(num_train):
        scores = X[i].dot(W) # scores.shape -->(1,C) or should say, (C,)?
        correct_class_score = scores[y[i]]
        num_high_margin = 0
        
        for j in range(num_classes):
            if j == y[i]:
                continue
                
            margin = scores[j] - correct_class_score + delta
#             print('margin_sample{0},class{1}: {2}'.format(i,j,margin)) #debug
            if margin > 0:
                loss += margin
                dW.T[j] += X[i]
                num_high_margin += 1
        
#         print('loss after loop{0}: {1}'.format(i,loss)) #debug
        
        dW.T[y[i]] += - num_high_margin * X[i]
#         print('dW after loop{1}: {0}'.format(dW,i)) #debug
  
  # Compute the gradient (analytic gradient) of the loss function and 
  # store it dW.     
  # it may be simpler to compute the derivative at the same time that
  # the loss is being computed.    

# Right now the loss is a sum over all training examples, but we want it
# to be an average instead so we divide by num_train. --> 'data loss'
# L=1/N * ∑i ∑j≠y[i] [max(0, f(x_i;W)_j - f(x_i, W)_y[j] + Δ]
# f(x_i;W)= Wx_i
    loss /= num_train
    dW /= num_train

# Add regularization, R(W)), to the loss. R(W)=∑k∑l(W_(k,l))^2
#     print(reg * np.sum(W*W)) #debug
    loss += reg * np.sum(W * W)
    dW += reg * np.sum(W * W)
#     print('final loss: {0}'.format(loss))
#     print('final dW: {0}'.format(dW))

    return loss, dW

In [47]:
def svm_loss_vectorized(X, y, W, reg):
    """
    A faster half-vectorized implementation. half-vectorized
  refers to the fact that for a single example the implementation contains
  no for loops, but there is still one loop over the examples (outside 
  this function)

  Inputs and outputs are the same as svm_loss_naive.
    """
    loss = 0.0
    dW = np.zeros(W.shape) #dW.shape = (num_total_pixels+1, num_classes)
    delta = 1.0
    scores = X.dot(W) # scores.shape = (len(X[0]), num_classes)
    correct_scores = scores[np.arange(X.shape[0]),y]
    num_classes = W.shape[1]
    num_train = X.shape[0]
    
    # Implement a vectorized version of the structured SVM loss, storing 
    # the result in loss.   
    
    margins = np.maximum(0, ((scores.T - correct_scores).T) + delta)
    # margins.shape = (len(X[0]), num_classes), same as scores.shape
#     print('margins matrix before y: {0}'.format(margins))
    margins[np.arange(X.shape[0]),y] = 0
#     print('margins matrix: {0}'.format(margins)) #debug
    loss = np.sum(margins)/num_train + reg * np.sum(W * W)
#     print('loss: {0}'.format(loss)) #debug
    margins[margins >0] = 1
    margins[np.arange(X.shape[0]),y] = - margins.sum(axis = 1)   
    dW = X.T.dot(margins)/num_train + reg * np.sum(W * W)
#     print('dW: {0}'.format(dW))
    
    # Implement a vectorized version of the gradient for the structured 
    # SVM loss, storing the result in dW. 

    return loss, dW  

### next cell for debug only

In [35]:
################################## Debug ###############################

# ### Load the raw CIFAR-10 data.
# cifar10_dir = '/Users/xiaoxiaoma/cifar-10-batches-py'

# # Cleaning up variables to prevent loading data multiple times (which \
# # may cause memory issue)
# try:
#     del X_train, y_train
#     del X_test, y_test
#     print('Clear previously loaded data.')
# except:
#     pass
# X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# mask_trial = 1
# X_trial = X_train[mask]
# y_trial = y_train[mask]

# X_trial = np.reshape(X_trial, (X_trial.shape[0], -1))
# X_trial -= mean_image
# X_trial = np.hstack([X_trial, np.ones((X_trial.shape[0], 1))])

# X_trial = X_trial[:5]
# y_trial = y_trial[:5]

# W_trial = init_W(X_trial, y_trial)

# loss, grad = svm_loss_naive(X_trial, y_trial, W_trial, 0.000005)
# loss_v, grad_v = svm_loss_vectorized(X_trial, y_trial, W_trial, 0.000005)

########################### Debug done ###########################

Clear previously loaded data.


In [49]:
W_dev = init_W(X_dev, y_dev)

In [50]:
loss, grad = svm_loss_naive(X_dev, y_dev, W_dev, 0.000005)
print('loss: %f' % (loss, ))
print(grad.shape)
print('grad: {0}'.format(grad[:1, :3]))

loss_v, grad_v = svm_loss_vectorized(X_dev, y_dev, W_dev, 0.000005)
print('loss_v: %f' % (loss_v, ))
print(grad_v.shape)
print('grad_v: {0}'.format(grad_v[:1, :3]))

loss: 8.937311
(3073, 10)
grad: [[  8.1696 -35.54    26.5696]]
loss_v: 8.937311
(3073, 10)
grad_v: [[  8.1696 -35.54    26.5696]]


In [52]:
# Numerically compute the gradient along several randomly chosen 
# dimensions, and compare them with your analytically computed gradient. 
# The numbers should match almost exactly along all dimensions.

from cs231n.gradient_check import grad_check_sparse
f = lambda w: svm_loss_naive(X_dev, y_dev, w, 0.0)[0]
grad_numerical = grad_check_sparse(f, W_dev, grad)

numerical: 4.536000 analytic: 4.536000, relative error: 1.352397e-11
numerical: 17.495200 analytic: 17.495200, relative error: 4.900638e-12
numerical: 16.870400 analytic: 16.870400, relative error: 2.575498e-12
numerical: 19.608400 analytic: 19.608400, relative error: 1.520399e-12
numerical: -53.580000 analytic: -53.580000, relative error: 1.426589e-12
numerical: 19.720800 analytic: 19.720800, relative error: 1.396707e-12
numerical: 52.612800 analytic: 52.612800, relative error: 2.500475e-13
numerical: -10.980000 analytic: -10.980000, relative error: 5.339505e-12
numerical: -24.112800 analytic: -24.112800, relative error: 2.115321e-12
numerical: 15.027600 analytic: 15.027600, relative error: 9.387944e-13


In [53]:
# do the gradient check once again with regularization turned on
# you didn't forget the regularization gradient did you?

loss, grad = svm_loss_naive(X_dev, y_dev, W_dev, 5e1)
f = lambda w: svm_loss_naive(X_dev, y_dev, w, 5e1)[0]
grad_numerical = grad_check_sparse(f, W_dev, grad)
        

numerical: -7.645085 analytic: -7.645682, relative error: 3.900987e-05
numerical: -25.999178 analytic: -26.000882, relative error: 3.277269e-05
numerical: -55.850053 analytic: -55.849682, relative error: 3.321933e-06
numerical: 21.337010 analytic: 21.336318, relative error: 1.621778e-05
numerical: 29.214354 analytic: 29.213118, relative error: 2.115546e-05
numerical: 98.709514 analytic: 98.709918, relative error: 2.050193e-06
numerical: -11.131357 analytic: -11.134882, relative error: 1.583114e-04
numerical: -12.157678 analytic: -12.154882, relative error: 1.149944e-04
numerical: -21.146756 analytic: -21.151282, relative error: 1.070070e-04
numerical: 7.694192 analytic: 7.693118, relative error: 6.974952e-05


In [54]:
# Next implement the function svm_loss_vectorized; for now only compute 
# the loss; we will implement the gradient in a moment.
tic = time.time()
loss_naive, grad_naive = svm_loss_naive(X_dev, y_dev, W_dev,0.000005)
toc = time.time()
print('Naive loss: %e computed in %fs' % (loss_naive, toc - tic))

tic = time.time()
loss_vectorized, grad_vectorized = svm_loss_vectorized(X_dev, y_dev, \
                                                       W_dev, 0.000005)
toc = time.time()
print('Vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# The losses should match but your vectorized implementation should be \
# much faster.
print('difference: %f' % (loss_naive - loss_vectorized))

Naive loss: 8.937311e+00 computed in 0.008822s
Vectorized loss: 8.937311e+00 computed in 0.001195s
difference: -0.000000


In [55]:
# Complete the implementation of svm_loss_vectorized, and compute the 
# gradient of the loss function in a vectorized way.

# The naive implementation and the vectorized implementation should match, 
# but the vectorized version should still be much faster.
tic = time.time()
_, grad_naive = svm_loss_naive(X_dev,y_dev,W_dev,0.000005)
toc = time.time()
print('Naive loss and gradient: computed in %fs' % (toc - tic))
print(grad_naive)

tic = time.time()
_, grad_vectorized = svm_loss_vectorized(X_dev, y_dev, \
                                         W_dev,0.000005)
toc = time.time()
print('Vectorized loss and gradient: computed in %fs' % (toc - tic))
print(grad_vectorized)

# The loss is a single number, so it is easy to compare the values 
# computed by the two implementations. The gradient on the other hand is 
# a matrix, so we use the Frobenius norm to compare them.
difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('difference: %f' % difference)

Naive loss and gradient: computed in 0.011329s
[[ 8.16960000e+00 -3.55400000e+01  2.65696000e+01 ...  9.24472000e+01
  -4.94000000e+00 -7.88496000e+01]
 [ 2.21000000e+00 -4.08200000e+01  3.04100000e+01 ...  9.45400000e+01
  -1.96200000e+01 -1.00450000e+02]
 [-1.15524000e+01 -2.99000000e+01  4.28476000e+01 ...  1.19903200e+02
  -4.29000000e+01 -1.24647600e+02]
 ...
 [ 4.93520000e+00 -6.08600000e+01  1.95352000e+01 ...  2.67464000e+01
   6.54000000e+00  2.47448000e+01]
 [ 1.48000005e-02 -6.74600000e+01  1.90148000e+01 ...  5.19736000e+01
  -4.66000000e+00  2.58652000e+01]
 [ 6.00000001e-01  5.11827353e-10  6.00000001e-01 ... -7.99999999e-01
   5.11827353e-10 -5.99999999e-01]]
Vectorized loss and gradient: computed in 0.001037s
[[ 8.16960000e+00 -3.55400000e+01  2.65696000e+01 ...  9.24472000e+01
  -4.94000000e+00 -7.88496000e+01]
 [ 2.21000000e+00 -4.08200000e+01  3.04100000e+01 ...  9.45400000e+01
  -1.96200000e+01 -1.00450000e+02]
 [-1.15524000e+01 -2.99000000e+01  4.28476000e+01 ...  

### Stochastic Gradient Descent

In [64]:
cd Documents/GitHub/machinelearningbasics/Stanfordcs231n/

/Users/xiaoxiaoma/Documents/GitHub/machinelearningbasics/Stanfordcs231n


In [65]:
import import_ipynb
import LinearClassifier

In [ ]:
svm = LinearSVM()
tic = time.time()
loss_hist = svm.train(X_train, y_train, learning_rate=1e-7, reg=2.5e4,
                      num_iters=1500, verbose=True)
toc = time.time()
print('That took %fs' % (toc - tic))

plt.plot(loss_hist)
plt.xlabel('Iteration number')
plt.ylabel('Loss value')
plt.show()

In [ ]:
# Write the LinearSVM.predict function and evaluate the performance on both the
# training and validation set
y_train_pred = svm.predict(X_train)
print('training accuracy: %f' % (np.mean(y_train == y_train_pred), ))
y_val_pred = svm.predict(X_val)
print('validation accuracy: %f' % (np.mean(y_val == y_val_pred), ))

In [ ]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of about 0.4 on the validation set.
learning_rates = [1e-7, 5e-5]
regularization_strengths = [2.5e4, 5e4]

# results is dictionary mapping tuples of the form
# (learning_rate, regularization_strength) to tuples of the form
# (training_accuracy, validation_accuracy). The accuracy is simply the fraction
# of data points that are correctly classified.
results = {}
best_val = -1   # The highest validation accuracy that we have seen so far.
best_svm = None # The LinearSVM object that achieved the highest validation rate.

################################################################################
# TODO:                                                                        #
# Write code that chooses the best hyperparameters by tuning on the validation #
# set. For each combination of hyperparameters, train a linear SVM on the      #
# training set, compute its accuracy on the training and validation sets, and  #
# store these numbers in the results dictionary. In addition, store the best   #
# validation accuracy in best_val and the LinearSVM object that achieves this  #
# accuracy in best_svm.                                                        #
#                                                                              #
# Hint: You should use a small value for num_iters as you develop your         #
# validation code so that the SVMs don't take much time to train; once you are #
# confident that your validation code works, you should rerun the validation   #
# code with a larger value for num_iters.                                      #
################################################################################
# Your code
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# Visualize the cross-validation results
import math
x_scatter = [math.log10(x[0]) for x in results]
y_scatter = [math.log10(x[1]) for x in results]

# plot training accuracy
marker_size = 100
colors = [results[x][0] for x in results]
plt.subplot(2, 1, 1)
plt.scatter(x_scatter, y_scatter, marker_size, c=colors)
plt.colorbar()
plt.xlabel('log learning rate')
plt.ylabel('log regularization strength')
plt.title('CIFAR-10 training accuracy')

# plot validation accuracy
colors = [results[x][1] for x in results] # default size of markers is 20
plt.subplot(2, 1, 2)
plt.scatter(x_scatter, y_scatter, marker_size, c=colors)
plt.colorbar()
plt.xlabel('log learning rate')
plt.ylabel('log regularization strength')
plt.title('CIFAR-10 validation accuracy')
plt.show()

In [ ]:
# Evaluate the best svm on test set
y_test_pred = best_svm.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('linear SVM on raw pixels final test set accuracy: %f' % test_accuracy)

In [ ]:
# Visualize the learned weights for each class.
# Depending on your choice of learning rate and regularization strength, these may
# or may not be nice to look at.
w = best_svm.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)
w_min, w_max = np.min(w), np.max(w)
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
      
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])